# How to use the kube client in api server
The existing workbench leveraged `ReplicationController`(https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller/). At this point, a `Deployment` that configures a `ReplicaSet` is now the recommended way to set up replication.
https://kubernetes.io/docs/concepts/workloads/controllers/deployment/
- [Access Clusters Using the Kubernetes API](https://kubernetes.io/docs/tasks/administer-cluster/access-cluster-api/)
- [Python Client github](https://github.com/kubernetes-client/python)
- [Python Client readthedocs](https://readthedocs.org/projects/kubernetes/)

## Check running kube cluster

In [1]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s\t%s" % (i.status.pod_ip, i.status.phase, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
172.17.0.8	Running	default	owncloud-deployment-7dc9db4f94-4l5cf
172.17.0.2	Running	kube-system	coredns-74ff55c5b-l6t4s
192.168.64.4	Running	kube-system	etcd-minikube
172.17.0.5	Running	kube-system	ingress-nginx-controller-65cf89dc4f-s2t59
192.168.64.4	Running	kube-system	kube-apiserver-minikube
192.168.64.4	Running	kube-system	kube-controller-manager-minikube
192.168.64.4	Running	kube-system	kube-proxy-swdlv
192.168.64.4	Running	kube-system	kube-scheduler-minikube
172.17.0.7	Running	kube-system	nfs-provisioner-76b9b4576-9gl9h
None	Failed	kube-system	nfs-provisioner-76b9b4576-d5rkh
192.168.64.4	Running	kube-system	storage-provisioner
172.17.0.4	Running	kubernetes-dashboard	dashboard-metrics-scraper-f6647bd8c-cxbmn
172.17.0.3	Running	kubernetes-dashboard	kubernetes-dashboard-968bcb79-gc4xp


In [2]:
from kubernetes import client, config


def main():
    # Configs can be set in Configuration class directly or using helper
    # utility. If no argument provided, the config will be loaded from
    # default location.
    config.load_kube_config()

    print("Supported APIs (* is preferred version):")
    print("%-40s %s" %
          ("core", ",".join(client.CoreApi().get_api_versions().versions)))
    for api in client.ApisApi().get_api_versions().groups:
        versions = []
        for v in api.versions:
            name = ""
            if v.version == api.preferred_version.version and len(
                    api.versions) > 1:
                name += "*"
            name += v.version
            versions.append(name)
        print("%-40s %s" % (api.name, ",".join(versions)))


if __name__ == '__main__':
    main()

Supported APIs (* is preferred version):
core                                     v1
apiregistration.k8s.io                   *v1,v1beta1
apps                                     v1
events.k8s.io                            *v1,v1beta1
authentication.k8s.io                    *v1,v1beta1
authorization.k8s.io                     *v1,v1beta1
autoscaling                              *v1,v2beta1,v2beta2
batch                                    *v1,v1beta1
certificates.k8s.io                      *v1,v1beta1
networking.k8s.io                        *v1,v1beta1
extensions                               v1beta1
policy                                   v1beta1
rbac.authorization.k8s.io                *v1,v1beta1
storage.k8s.io                           *v1,v1beta1
admissionregistration.k8s.io             *v1,v1beta1
apiextensions.k8s.io                     *v1,v1beta1
scheduling.k8s.io                        *v1,v1beta1
coordination.k8s.io                      *v1,v1beta1
node.k8s.io            

## How to create a Deployment
- https://github.com/kubernetes-client/python/blob/master/examples/notebooks/create_deployment.ipynb

### 1. Load config

In [3]:
from kubernetes import client, config

LABELS = {"app": "nginx"}
DEPLOYMENT_NAME = "nginx-deployment"

# Load config from default location
config.load_kube_config()
apps_api = client.AppsV1Api()

apps_api

### 2. Pod container

In [4]:
# Configure Pod template container
container = client.V1Container(
    name = "nginx",
    image="nginx:1.15.4",
    ports=[client.V1ContainerPort(container_port=80)],
    resources=client.V1ResourceRequirements(
        requests={"cpu": "100m", "memory":"200Mi"},
        limits={"cpu": "500m", "memory":"500Mi"},
    ),
)

#container

### 3. Spec section

In [5]:
template = client.V1PodTemplateSpec(
    metadata=client.V1ObjectMeta(labels=LABELS),
    spec=client.V1PodSpec(containers=[container]),
)

#template

### 4. Specification of deployment

In [6]:
spec = client.V1DeploymentSpec(
    replicas=1,
    template=template,
    selector=client.V1LabelSelector(match_labels=LABELS)
)

#spec

### 5. Instantiate the deployment object

In [7]:
deployment = client.V1Deployment(
    api_version="apps/v1",
    kind="Deployment",
    metadata=client.V1ObjectMeta(name=DEPLOYMENT_NAME),
    spec=spec,
)

#deployment

### 6. Create deployment

In [8]:
def create_deployment(api, deployment):
    # Create deployement
    resp = api.create_namespaced_deployment(
        body=deployment, namespace="default"
    )

    print("\n[INFO] deployment `nginx-deployment` created.\n")
    print("%s\t%s\t\t\t%s\t%s" % ("NAMESPACE", "NAME", "REVISION", "IMAGE"))
    print(
        "%s\t\t%s\t%s\t\t%s\n"
        % (
            resp.metadata.namespace,
            resp.metadata.name,
            resp.metadata.generation,
            resp.spec.template.spec.containers[0].image,
        )
    )


In [9]:
create_deployment(apps_api, deployment)


[INFO] deployment `nginx-deployment` created.

NAMESPACE	NAME			REVISION	IMAGE
default		nginx-deployment	1		nginx:1.15.4



### 7. Delete deployment

In [ ]:
def delete_deployment(api):
    # Delete deployment
    resp = api.delete_namespaced_deployment(
        name=DEPLOYMENT_NAME,
        namespace="default",
        body=client.V1DeleteOptions(
            propagation_policy="Foreground", grace_period_seconds=5
        ),
    )
    print("\n[INFO] deployment `nginx-deployment` deleted.")


In [ ]:
delete_deployment(apps_api)

## How to create Service
- https://github.com/kubernetes-client/python/blob/master/examples/notebooks/create_service.ipynb
### 1. Load config

In [10]:
from kubernetes import client, config

LABELS = {"app": "nginx"}
SERVICE_NAME = "nginx-service"

config.load_kube_config()
api_instance = client.CoreV1Api()
service = client.V1Service()

service

{'api_version': None,
 'kind': None,
 'metadata': None,
 'spec': None,
 'status': None}

### 2. Create API resource instance with Service .spec description
We define `NodePort` to expose the service.Then we can access this service with `http://<minikube ip>:<node_port>`.

For example, run the following command in the terminal running minikube.
```
% minikube ip
192.168.64.4
```
Then you can access `http://192.168.64.4:32000/` 

In [11]:
service.api_version = "v1"
service.kind = "Service"
service.metadata = client.V1ObjectMeta(name=SERVICE_NAME)

spec = client.V1ServiceSpec()
spec.selector = LABELS
spec.type = "NodePort"
spec.ports = [client.V1ServicePort(protocol="TCP", port=8080, target_port=80, node_port=32000)]
service.spec = spec

#service

### 3. Create Service

In [12]:
api_instance.create_namespaced_service(namespace="default", body=service)

{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2021, 5, 31, 21, 4, 28, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': [{'api_version': 'v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:externalTrafficPolicy': {},
                                                           'f:ports': {'.': {},
                                                                       'k:{"port":8080,"protocol":"TCP"}': {'.': {},
                                                                                                            'f:nodePort': {},
                                 

### 4. Delete Service

In [ ]:
api_instance.delete_namespaced_service(name=SERVICE_NAME, namespace="default")

## Example: owncloud
- Dependancy - How to run clowder with mongo using Python client?
- Need to prepare volumn? before running pods, user volume need to be prepared.

### 1. Load the ndslab json

In [13]:
import json

with open('owncloud.json') as json_file:
    nds_spec = json.load(json_file)

nds_spec

{'key': 'owncloud',
 'label': 'ownCloud',
 'description': 'A self-hosted file sync and share server',
 'logo': '/asset/png/logos/owncloud-logo.png',
 'maintainer': '',
 'image': {'registry': '', 'name': 'ndslabs/owncloud', 'tags': ['latest']},
 'display': 'stack',
 'access': 'external',
 'depends': [{'key': 'mysql', 'required': False, 'shareConfig': False},
  {'key': 'postgres', 'required': False, 'shareConfig': False}],
 'ports': [{'port': 80, 'protocol': 'http'}],
 'repositories': [{'url': 'https://github.com/owncloud/core', 'type': 'git'}],
 'readinessProbe': {'type': 'http',
  'path': '/core/img/favicon.ico',
  'port': 80,
  'initialDelay': 15,
  'timeout': 120},
 'volumeMounts': [{'type': '', 'mountPath': '/var/www/owncloud'}],
 'resourceLimits': {'cpuMax': 500,
  'cpuDefault': 100,
  'memMax': 1000,
  'memDefault': 50},
 'developerEnvironment': 'cloud9php',
 'tags': ['6', '11'],
 'info': 'https://nationaldataservice.atlassian.net/wiki/display/NDSC/ownCloud'}

### 2. Load config for deployment

In [14]:
from kubernetes import client, config

LABELS = {"app": nds_spec['label']}
DEPLOYMENT_NAME = nds_spec['key']+"-deployment"
SERVICE_NAME = nds_spec['key']+"-service"

# Load config from default location
config.load_kube_config()
apps_api = client.AppsV1Api()

### 3. Volume
We can use the persistent volume, not an `emptyDir` volume in this example.

In [15]:
USER_ID = 'dynam'
VOLUME_NAME = '{}-storage'.format(USER_ID)

volume = client.V1Volume(
    name = VOLUME_NAME,
    empty_dir = client.V1EmptyDirVolumeSource(),
)

### 4. Pod container

In [16]:
# Configure Pod template container
container = client.V1Container(
    name = nds_spec['key'],
    image= nds_spec['image']['name']+":"+nds_spec['image']['tags'][0],
    ports=[client.V1ContainerPort(container_port=nds_spec['ports'][0]['port'])],
    resources=client.V1ResourceRequirements(
        requests={"cpu": "{}m".format(nds_spec['resourceLimits']['cpuDefault']), 
                  "memory":"{}Mi".format(nds_spec['resourceLimits']['memDefault'])},
        limits={"cpu": "{}m".format(nds_spec['resourceLimits']['cpuMax']), 
                "memory":"{}Mi".format(nds_spec['resourceLimits']['memMax'])},
    ),
    volume_mounts=[client.V1VolumeMount(name=VOLUME_NAME, mount_path=nds_spec['volumeMounts'][0]['mountPath'])],
)

#container

### 5. Define Deployment

In [17]:
template = client.V1PodTemplateSpec(
    metadata=client.V1ObjectMeta(labels=LABELS),
    spec=client.V1PodSpec(containers=[container], volumes=[volume]),
)

spec = client.V1DeploymentSpec(
    replicas=1,
    template=template,
    selector=client.V1LabelSelector(match_labels=LABELS)
)

deployment = client.V1Deployment(
    api_version="apps/v1",
    kind="Deployment",
    metadata=client.V1ObjectMeta(name=DEPLOYMENT_NAME),
    spec=spec,
)

### 5. Create Deployment

In [18]:
def create_deployment(api, deployment):
    # Create deployement
    resp = api.create_namespaced_deployment(
        body=deployment, namespace="default"
    )

    print("\n[INFO] deployment `{}` created.\n".format(DEPLOYMENT_NAME))
    print("%s\t%s\t\t\t%s\t%s" % ("NAMESPACE", "NAME", "REVISION", "IMAGE"))
    print(
        "%s\t\t%s\t%s\t\t%s\n"
        % (
            resp.metadata.namespace,
            resp.metadata.name,
            resp.metadata.generation,
            resp.spec.template.spec.containers[0].image,
        )
    )

def delete_deployment(api):
    # Delete deployment
    resp = api.delete_namespaced_deployment(
        name=DEPLOYMENT_NAME,
        namespace="default",
        body=client.V1DeleteOptions(
            propagation_policy="Foreground", grace_period_seconds=5
        ),
    )
    print("\n[INFO] deployment `{}` deleted.".format(DEPLOYMENT_NAME))

In [19]:
create_deployment(apps_api, deployment)


[INFO] deployment `owncloud-deployment` created.

NAMESPACE	NAME			REVISION	IMAGE
default		owncloud-deployment	1		ndslabs/owncloud:latest



### 6. Create Service
After creating service, we can access `owncloud` with `http://192.168.64.4:32001/`.

In [20]:
api_instance = client.CoreV1Api()

service = client.V1Service()
service.api_version = "v1"
service.kind = "Service"
service.metadata = client.V1ObjectMeta(name=SERVICE_NAME)

spec = client.V1ServiceSpec()
spec.selector = LABELS
spec.type = "NodePort"
spec.ports = [client.V1ServicePort(protocol="TCP", port=8080, target_port=80, node_port=32001)]
service.spec = spec

api_instance.create_namespaced_service(namespace="default", body=service)

{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2021, 5, 31, 21, 5, 2, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': [{'api_version': 'v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:externalTrafficPolicy': {},
                                                           'f:ports': {'.': {},
                                                                       'k:{"port":8080,"protocol":"TCP"}': {'.': {},
                                                                                                            'f:nodePort': {},
                                  

## Define the persistent volume (pv) and persistent volume claim (pvc)
- In this example, we define 'Local storage'. Please see https://kubernetes.io/docs/concepts/storage/volumes/ for volumes in detail.
### 1. Persistent volume

In [21]:
from kubernetes import client

USER_ID = 'dynam'
pv_name = "local-pv-{}".format(USER_ID)

pv = client.V1PersistentVolume(
    api_version="v1",
    kind="PersistentVolume",
    metadata=client.V1ObjectMeta(
        name=pv_name,
        labels={'app': pv_name}
    ),
    spec=client.V1PersistentVolumeSpec(
        access_modes=["ReadWriteMany"],
        capacity={'storage': '10Gi'},
        volume_mode='Filesystem',
        storage_class_name="local-storage",
        local=client.V1LocalVolumeSource(
            path='/mnt/persistent-volume'
        ),
        node_affinity=client.V1VolumeNodeAffinity(
            required=client.V1NodeSelector(
                node_selector_terms=[
                    client.V1NodeSelectorTerm(
                        match_expressions=[
                            client.V1NodeSelectorRequirement(
                                key='kubernetes.io/hostname',
                                operator='In',
                                values=['minikube']
                            )
                        ]
                    )
                ]
            )
        ),
    )
)

api_instance = client.CoreV1Api()

try:
    api_instance.create_persistent_volume(pv)
except client.rest.ApiException as e:
    print("Got exception: while creating the LOCAL PV", e)

### 2. Persistent volume claim

In [22]:
pvc_name = "local-pvc-{}".format(USER_ID)

pvc = client.V1PersistentVolumeClaim(
    api_version="v1",
    kind="PersistentVolumeClaim",
    metadata=client.V1ObjectMeta(
        name=pvc_name,
        labels={'app': pv_name}
    ),
    spec=client.V1PersistentVolumeClaimSpec(
        access_modes=["ReadWriteMany"],
        resources=client.V1ResourceRequirements(
            requests={'storage': '10Gi'}
        ),
        selector=client.V1LabelSelector(
            match_labels={'app': pv_name}
        ),
        storage_class_name="local-storage"
    )
)

try:
    api_instance.create_namespaced_persistent_volume_claim('default',pvc)
except client.rest.ApiException as e:
    print("Got exception: %s\n while creating the LOCAL PVC", e)

## Print the list of pvc

In [23]:
from kubernetes import client, config

config.load_kube_config()
api_instance = client.CoreV1Api()

pvcs = api_instance.list_namespaced_persistent_volume_claim(
      namespace="default", watch=False)

#pvcs

In [24]:
print("---- PVCs ---")
print("%-16s\t%-40s\t%-6s" % ("Name", "Volume", "Size"))
for pvc in pvcs.items:
    print("%-16s\t%-40s\t%-6s" %
        (pvc.metadata.name, pvc.spec.volume_name,    
        pvc.spec.resources.requests['storage']))

---- PVCs ---
Name            	Volume                                  	Size  
local-pvc-dynam 	local-pv-dynam                          	10Gi  
